In [22]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np

#caminho onde estão os pesos
face_classifier = cv2.CascadeClassifier('cascade/haarcascade_frontalface_default.xml') 
eyes_classifier = cv2.CascadeClassifier('cascade/haarcascade_mcs_eyepair_big.xml') 

 ## **R1 - NOTA 6**

Faz detecção de face em tempo real em vídeo ou webcam. Como resultado deve ter 2 janelas. A primeira janela com a imagem original e a segunda com a filtragem apenas na área da face.
Opções de filtros para aplicar na área da face:
- Filtro linear como filtro de borramento E/OU filtro de realce E/OU filtro de contorno E/OU qualquer outro filtro de sua escolha. 
Note que é necessário aplicar pelo menos uma das técnicas de filtragem em um vídeo e apenas na área da face. Implementações extras, valem como nota extra...

In [23]:
def linear_filter(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_return = face_classifier.detectMultiScale(img_gray, scaleFactor = 1.2, minNeighbors = 5)
    if len(faces_return) != 0:     
         for (x,y,w,h) in faces_return:
            face_area = img[y:y+h, x:x+w]

            ##Aplica filtro linear de blur e medianBlur
            img[y:y+h, x:x+w] = cv2.blur(face_area, (80, 80))
            img[y:y+h, x:x+w] = cv2.medianBlur(face_area, 25)

            ##Aplica filtro de contorno
            thresh = cv2.adaptiveThreshold(img_gray[y:y+h, x:x+w],255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,23,3)
            cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            for c in cnts:
                area = cv2.contourArea(c)
                if area > 10:
                    cv2.drawContours(img[y:y+h, x:x+w], [c], -1, (36,255,12), 1)

         return img
    else:
       return img

vc = cv2.VideoCapture(0)

if vc.isOpened(): 
    rval, frame = vc.read()

else:
    rval = False

while rval:
    cv2.imshow("original", frame)

    img = linear_filter(frame)
    cv2.imshow("preview", img)


    rval, frame = vc.read()
    key = cv2.waitKey(20)
    if key == 27: 
        break
vc.release()
cv2.destroyAllWindows()

## **R2 - NOTA 8** 
Faz detecção de face em tempo real em vídeo ou webcam. Como resultado deve ter 2 janelas. A primeira janela com a imagem original e a segunda com filtro de sobreposição.
Opções de filtros para aplicar sobre a face:
- Na pasta img estão os exemplos de imagens que podem ser utilizadas tais como: dog, bigode, chapéu.
Deve ser utilizado pelo menos uma das imagens para criar esse efeito. Usa o bigode OU(OR) chapéu OU(OR) …. 
Importante! A imagem de saída deve fazer sentido, ou seja, o chapéu na cabeça, o óculos sobre os olhos… um bigode na testa não faz sentido….Implementações extras, valem como nota extra...

In [24]:
glasses = cv2.imread("img/glasses.png", -1)

def overlay_filter(img):
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(img_gray, scaleFactor=1.5, minNeighbors=5)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)

    for (x, y, w, h) in faces:
        roi_gray    = img_gray[y:y+h, x:x+w] 
        roi_color   = img[y:y+h, x:x+w]

        eyes = eyes_classifier.detectMultiScale(roi_gray, scaleFactor=1.5, minNeighbors=5)

        for (ex, ey, ew, eh) in eyes:
            roi_eyes = roi_gray[ey:ey+eh, ex:ex+ew]
            resized_glasses = cv2.resize(glasses.copy(), (ew, eh))

            gw, gh, gc = resized_glasses.shape
            for i in range(0, gw):
                for j in range(0, gh):
                    if resized_glasses[i, j][3] != 0:
                        roi_color[ey+i, ex+j] = resized_glasses[i, j]

    return img   

         
vc = cv2.VideoCapture(0)

if vc.isOpened(): 
    rval, frame = vc.read()

else:
    rval = False

while rval:
    cv2.imshow("original", frame)

    img = overlay_filter(frame)
    cv2.imshow("preview", img)


    rval, frame = vc.read()
    key = cv2.waitKey(20)
    if key == 27: 
        break
vc.release()
cv2.destroyAllWindows()

## **R3 - NOTA 10**
O aluno(a) pode alcançar a rubrica R3 ,com nota 10, de algumas formas diferente:
- OU(OR) Faz o R1 E(AND) o R2 juntos dependendo do click do mouse. Realiza os dois processos de filtragem um de cada vez, para alternar entre os dois, o usuário faz um click com o mouse. O programa começa sem aplicar filtragem, quando é feito um click esquerdo com o mouse, realiza R1, no segundo click, realiza R2, no terceiro click volta para R1, e assim por diante...quando tem click do botão direito volta a imagem sem filtro. 

- OU(OR)  Faz o R1 com mais de 3 filtros diferentes, dependendo do click do mouse. Realiza os dois processos de filtragem um de cada vez, para alternar entre os três, o usuário faz um click com o mouse. O programa começa sem aplicar filtragem, quando é feito um click esquerdo com o mouse, aplicar filtro 1, no segundo click,  aplicar filtro 2, no terceiro click,  aplicar filtro 3, no quarto volta para filtro 1, e assim por diante...quando tem click do botão direito volta a imagem sem filtro.

- OU(OR) Faz o R2 com mais de 3 filtros diferentes, dependendo do click do mouse. Realiza os dois processos de filtragem um de cada vez, para alternar entre os três, o usuário faz um click com o mouse. O programa começa sem aplicar filtragem, quando é feito um click esquerdo com o mouse, aplicar filtro 1, no segundo click,  aplicar filtro 2, no terceiro click,  aplicar filtro 3, no quarto volta para filtro 1, e assim por diante...quando tem click do botão direito volta a imagem sem filtro.


In [27]:
clicks = 1
rbutton = False

def mouse_click(event, x, y, flags, param):
    global clicks, rbutton

    if event == cv2.EVENT_RBUTTONDOWN:
        print(str('Clique no botão direito: filtro desativado'))
        rbutton = True
    if event == cv2.EVENT_LBUTTONDOWN:
         rbutton = False
         print(str(clicks) + 'º clique')
         clicks = clicks + 1


vc = cv2.VideoCapture(0)

if vc.isOpened(): 
    rval, frame = vc.read()

else:
    rval = False

while rval:

    cv2.imshow("original", frame)
    cv2.imshow("preview", frame)

    cv2.setMouseCallback('preview', mouse_click)
    
    if(rbutton or clicks == 1):
        cv2.imshow("preview", frame)
    elif(clicks % 2 == 0):
        cv2.imshow("preview", linear_filter(frame))
    else:
        cv2.imshow("preview", overlay_filter(frame))

    rval, frame = vc.read()
    key = cv2.waitKey(20)
    if key == 27: 
        break
    
vc.release()
cv2.destroyAllWindows()
    


1º clique
2º clique
3º clique
4º clique
5º clique
6º clique
7º clique
8º clique
9º clique
Clique no botão direito: filtro desativado
10º clique
